# Notebook with  MoveNet and YOLO and Classification

In [1]:
import torch, cv2
import numpy as np

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math

C:\Users\keven\anaconda3\envs\dlav2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
#load the model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

Using cache found in C:\Users\keven/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-22 torch 1.11.0 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [18]:
# model settings:
model.classes = [0] # only detect humans -> class = 0
model.max_det = 6

In [3]:
def get_info_objs(results):
    return results.pandas().xyxy[0].sort_values('confidence')

In [4]:
def display_image(img):
    cv2.imshow('hello',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def get_cropped_images(pd,frame): #pd is a 
    nb_detected_objs = len(pd.index)
    
    index = np.zeros((4,nb_detected_objs),dtype=int)#4 rows for x0,y0,x1,y1 --> each column is the bb of 1 img
    images = []
    
    counter = 0
    for obj in pd.iloc:
        x0, y0, x1, y1= obj.to_numpy()[0:4].astype(int)
        images.append(frame[y0:y1, x0:x1])
        index[:,counter] = np.array([x0,y0,x1,y1],dtype=int)
        counter += 1
        
    return images, index 

In [6]:
interpreter = tf.lite.Interpreter(model_path='C:\DLAV_testing\MoveNetLightning\lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [8]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [9]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

# Pose Classification

In [10]:
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1, y1 = landmark1
    x2, y2 = landmark2
    x3, y3 = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    angle = np.abs(angle)
    # Check if the angle is less than zero.
    if angle > 180.0:

        angle = 360-angle
    
    # Return the calculated angle.
    return angle

**Outputs**

A float32 tensor of shape [1, 1, 17, 3].

● The first two channels of the last dimension represents the yx coordinates (normalized to
image frame, i.e. range in [0.0, 1.0]) of the 17 keypoints (in the order of: 

**[0: nose, 1: left eye, 2: right eye, 3: left ear, 4:_right ear, 5: left shoulder, 6: right shoulder, 7: left elbow, 8: right elbow, 9: left wrist, 10:right wrist, 11: left hip, 12: right hip, 13: left knee, 14: right knee, 15: left ankle, 16: right ankle]).**

● The third channel of the last dimension represents the prediction confidence scores of
each keypoint, also in the range [0.0, 1.0].

In [25]:
def classifyPose(kp_array, output_image, display=False):
    '''
    This function classifies yoga poses depending upon the angles of various body joints.
    Args:
        kp_array: A list of detected landmarks of the person whose pose needs to be classified.
        output_image: A image of the person with the detected pose landmarks drawn.
        display: A boolean value that is if set to true the function displays the resultant image with the pose label 
        written on it and returns nothing.
    Returns:
        output_image: The image with the detected pose landmarks drawn and pose label written.
        label: The classified pose label of the person in the output_image.

    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(kp_array[5],
                                      kp_array[7],
                                      kp_array[9])
    
    # Get the angle between the right shoulder, elbow and wrist points.
    right_elbow_angle = calculateAngle(kp_array[6],
                                       kp_array[8],
                                       kp_array[10])
    
    # Get the angle between the left elbow, shoulder and hip points.
    left_shoulder_angle = calculateAngle(kp_array[7],
                                         kp_array[5],
                                         kp_array[11])
    # Get the angle between the right hip, shoulder and elbow points.
    right_shoulder_angle = calculateAngle(kp_array[12],
                                          kp_array[6],
                                          kp_array[8])
    
    #print('left_elbow_angle: ', left_elbow_angle ,'\n right_elbow_angle: ', right_elbow_angle)
    #print('left_shoulder_angle: ', left_shoulder_angle ,'\n right_shoulder_angle: ', right_shoulder_angle)
    
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the both arms are straight.
    if left_elbow_angle > 125 and left_elbow_angle < 220 and right_elbow_angle > 125 and right_elbow_angle < 220:
        #label = 'T Pose'
        # Check if shoulders are at the required angle.
        if left_shoulder_angle > 70 and left_shoulder_angle < 110 and right_shoulder_angle > 70 and right_shoulder_angle < 110:
            label = 'T Pose'
                        
    if right_elbow_angle > 50 and right_elbow_angle < 130 and right_shoulder_angle > 70 and right_shoulder_angle < 110:
        label = "power to the people"
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
    
    # Check if the resultant image is specified to be displayed.
    if display:
    
        # Display the resultant image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        
        # Return the output image and the classified label.
        return output_image, label

In [26]:
def write_frame_keypoints(imglist,indexes,frame): #write keypoints and classify on each cropped image and fuse it with the main frame
    for i,objs in enumerate(imglist):    
        img = objs
        img1 = img.copy()

        img1 = tf.image.resize_with_pad(np.expand_dims(img1, axis=0), 192,192)
        input_image = tf.cast(img1, dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

        # Rendering 
        draw_connections(img, keypoints_with_scores, EDGES, 0.1)
        draw_keypoints(img, keypoints_with_scores, 0.1)
        
        landmarks = keypoints_with_scores.reshape((17,3))[:,0:2]#array of landmarks (x,y)
        
        #Classification
        if np.size(landmarks) != 0:
            img, _ = classifyPose(landmarks, img, display=False)
        

        frame[indexes[1,i]:indexes[3,i], indexes[0,i]:indexes[2,i]] = img
        
    return frame

In [27]:
cap = cv2.VideoCapture(0)
#RUN ALL FUNCTIONS IN THIS NOTEBOOK BEFOREHAND!!!!

#cap = cv2.VideoCapture("C:\DLAV_testing\images to test\Business-people-dancing1.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue
        
    # Reshape image
    img = frame.copy()
    
    results = model(img)#from yolo
    
    pd = get_info_objs(results)
    
    imglist,indexes = get_cropped_images(pd,frame)
    
    img = write_frame_keypoints(imglist,indexes,frame)#movenet
    
    cv2.imshow('Yolo + Movenet + Classification', img)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [20]:
img = cv2.imread(r"C:\DLAV_testing\images to test\test_angle.jpg")

results = model(img)#from yolo
    
pd = get_info_objs(results)
    
imglist,indexes = get_cropped_images(pd,img)
    
img = write_frame_keypoints(imglist,indexes,img)#movenet
    
cv2.imshow('Yolo + Movenet + Classification', img)
    
cv2.waitKey(0)
cv2.destroyAllWindows()
        

